# The topological phase boundary in $E_Z$ - $V_G$ space

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plb
from cmath import sqrt
from scipy import optimize
import logging
import cPickle as pickle
import kwant
import dolfin
import electrostatics as estat
import generate_system
logging.getLogger('FFC').setLevel(logging.WARNING)

In [ ]:
"""Set system dimensions and constants"""

hbar = 1.055e-34
m = 9.109e-31
meff = 0.014*m
akwant, aFEM = 2.5e-9, 5e-9
eV, e = 1.6023e-19, 1.6023e-19
eps0 = 8.854e-12
gamma=hbar**2/(2*meff*akwant**2)
energy_par = estat.EnergyParameters(akwant=akwant, eps_dielec=8.0, eps_wire=17.7, 
                                    gamma=hbar**2/(2*meff*akwant**2), scaler=eV/gamma, 
                                    bandgap=0.1, Delta=0.5e-3, 
                                    poisson_scale=e*aFEM**2/(akwant**3*eps0),
                                    miliscaler = 1000*gamma/eV)

dimension_par = estat.DimensionParameters(radius=50e-9/aFEM, top=250e-9/aFEM, 
                                          top_sc=187e-9/aFEM - 10*np.sin(np.pi/3.0),
                                          side=300e-9/aFEM, dielec_thickness=30e-9/aFEM)
solver_kwargs = {'alpha': 0.5, 'M': 5, 'verbose': 0, 'f_tol': 1e-7, 'maxiter': 200} 

environment_parameters = [{'V_SC': 0.0, 'l_SO': 233e-9, 'tag': '0V, 233nm', 'Ez_init': 5e-3},
                        {'V_SC': 0.2, 'l_SO': 233e-9, 'tag': '02V, 233nm', 'Ez_init': 5e-3},
                        {'V_SC': 0.0, 'l_SO': 100e-9, 'tag': '0V, 100nm', 'Ez_init': 7e-3},
                        {'V_SC': 0.2, 'l_SO': 100e-9, 'tag': '02V, 100nm', 'Ez_init': 7e-3},
                        {'V_SC': 0.0, 'l_SO': 70e-9, 'tag': '0V, 70nm', 'Ez_init': 10e-3},
                        {'V_SC': 0.2, 'l_SO': 70e-9, 'tag': '02V, 70nm', 'Ez_init': 10e-3},
                        {'V_SC': 0.0, 'l_SO': 60e-9, 'tag': '0V, 60nm', 'Ez_init': 15e-3},
                        {'V_SC': 0.2, 'l_SO': 60e-9, 'tag': '02V, 60nm', 'Ez_init': 15e-3},
                        {'V_SC': 0.0, 'l_SO': 50e-9, 'tag': '0V, 50nm', 'Ez_init': 15e-3},
                        {'V_SC': 0.2, 'l_SO': 50e-9, 'tag': '02V, 50nm', 'Ez_init': 15e-3}]

In [ ]:
"""Define functions to compute the topological phase boundary"""

def samesign(a, b):
    return a * b > 0

def top_regime(Ez, mu, energy_par):
    Delta = energy_par.scaler*energy_par.Delta
    return Ez - sqrt(abs(mu)**2 + Delta**2).real

def bisect_pert(func, Ez_low, Ez_high, l_SO, modes, n, perturbations, 
                dmu_init, mu0, energies, V_G, energy_par):
    mu_low = -mu0 + func(modes, energies, n, perturbations, Ez_low, 
                         l_SO, dmu_init, V_G, energy_par)[0][-1]
    mu_high = -mu0 + func(modes, energies, n, perturbations, Ez_high, 
                          l_SO, dmu_init, V_G, energy_par)[0][-1]
    
    for i in range(100):
        Ez_mid = (Ez_low + Ez_high)/2.0
        dmu_mid = func(modes, energies, n, perturbations, Ez_mid, 
                       l_SO, dmu_init, V_G, energy_par)[0]
        mu_mid = -mu0 + dmu_mid[-1]
        dmu_init = dmu_mid
        func_mid = top_regime(Ez_mid, mu_mid, energy_par)
        func_low = top_regime(Ez_low, mu_low, energy_par)
        func_high = top_regime(Ez_high, mu_high, energy_par)
        if samesign(func_low, func_mid):
            Ez_low = Ez_mid
            mu_low = mu_mid
        else:
            Ez_high = Ez_mid
            mu_high = mu_mid
    return Ez_mid, mu_mid, dmu_init

def energy_lowerbound_range(V_G_range_0V, V_G_range_02V, V_SC):
    if V_SC == 0:
        V_G_range = V_G_range_0V
        if V_G_range[-1] > 0:
            energy_lowerbound = -eV/gamma*V_G_range[-1]
        else:
            energy_lowerbound = 0
    else:
        V_G_range = V_G_range_02V
        if V_SC > 0.3*V_G_range[-1]:
            energy_lowerbound = -eV/gamma*V_SC
        else:
            energy_lowerbound = -eV/gamma*V_G_range[-1]
    return energy_lowerbound, V_G_range

In [ ]:
sys = generate_system.kwant_hexagonal_nanowire(energy_par, dimension_par, a=0.5)
site_pos = np.array([sys.site(i).pos for i in xrange(sys.graph.num_nodes)])
mesh, mesh_points, length, length_wire = generate_system.mesh_hexagonal_nanowire(dimension_par)[:4]
V = dolfin.FunctionSpace(mesh, 'Lagrange', 1)
discr_par = estat.DiscretizationParameters(sys=sys, site_pos=site_pos, mesh=mesh, 
                                           mesh_points=mesh_points, length=length, 
                                           length_wire=length_wire, V=V)

V_G_range_0V = np.arange(0.285, 0.336, 0.001)[::-1]
V_G_range_02V = np.arange(-0.98, -0.80, 0.003)[::-1]

mu_topo_dict = {}
Ez_topo_dict = {}
n_topo_dict = {}

for environment in environment_parameters:
    V_SC = environment['V_SC']
    l_SO = environment['l_SO']
    alpha = akwant/l_SO
    Ez_init = energy_par.scaler*environment['Ez_init']
    mu_init = 0
    max_Ez = 20e-3*eV/gamma
    min_Ez = 0
    initial_guess = 0.06*np.ones(discr_par.length)
    dmu_init = 0.001
    Ez_lowerbound = 0.0
    Ez_upperbound = 20e-3*eV/gamma
    mu_topo_store = []
    Ez_topo_store = []
    n_topo_store = []

    
    sigma, V_G_range = energy_lowerbound_range(V_G_range_0V, V_G_range_02V, V_SC)
            
    for V_G in V_G_range:
        perturbations, mu0, modes, n, sp_sol, energies = \
        estat.init_field_simulation(Ez_init, initial_guess, sigma, l_SO, V_G, V_SC,
                                    generate_system.potential, solver_kwargs, energy_par, 
                                    dimension_par, discr_par)[:6]
        
        Ez_topo, mu_topo, dmu_init = bisect_pert(estat.self_consistent_mu, Ez_lowerbound, 
                                                 Ez_upperbound, l_SO, modes, n, perturbations, 
                                                 dmu_init, mu0, energies, V_G, energy_par)[:3]
        n_topo_store.append(n)
        mu_scaled = 1000*gamma/eV*mu_topo
        mu_init = mu_topo
        initial_guess = sp_sol
        Ez_topo_scaled = 1000*gamma/eV*Ez_topo
        mu_topo_store.append(mu_scaled)
        Ez_topo_store.append(Ez_topo_scaled)
    
    mu_topo_dict[environment['tag']] = mu_topo_store
    Ez_topo_dict[environment['tag']] = Ez_topo_store
    n_topo_dict[environment['tag']] = n_topo_store

In [ ]:
kw_str_0V = ['0V, 233nm', '0V, 100nm', '0V, 70nm', '0V, 60nm', '0V, 50nm']
kw_str_02V = ['02V, 233nm', '02V, 100nm', '02V, 70nm', '02V, 60nm', '02V, 50nm']
label_str = ['$l_{SO} = 233$ nm', '$l_{SO} = 100$ nm', '$l_{SO} = 70$ nm',\
             '$l_{SO} = 60$ nm', '$l_{SO} = 50$ nm']
clr_str = ['k--', 'r--', 'b-', 'k-', 'r-']

fig = plt.figure(figsize=(20,7))

ax1 = fig.add_subplot(121)
for i, kw in enumerate(kw_str_0V):
    ax1.plot(Ez_topo_dict[kw], V_G_range_0V, clr_str[i], label=label_str[i], lw=3)
ax1.set_xlabel('$E_Z$ [meV]', fontsize=24)
ax1.set_ylabel('$V_G [V]$', fontsize = 24)
ax1.text(0.2, 0.9, '$V_{SC} = 0$ V', transform=ax1.transAxes, fontsize=30)
ax1.legend(loc='center right', fontsize=17)
ax1.tick_params(labelsize=17)
ax1.locator_params(axis='x', nbins=5)
ax1.locator_params(axis='y', nbins=5)
ax1.set_xlim(0, 6)
ax1.set_ylim(0.285, 0.335)

ax2 = fig.add_subplot(122)
for i, kw in enumerate(kw_str_02V):
    ax2.plot(Ez_topo_dict[kw], V_G_range_02V, clr_str[i], label=label_str[i], lw=3)
ax2.set_xlabel('$E_Z$ [meV]', fontsize=24)
ax2.set_ylabel('$V_G [V]$', fontsize = 24)
ax2.text(0.2, 0.9, '$V_{SC} = 0.2$ V', transform=ax2.transAxes, fontsize=30)
ax2.set_xlim(0, 4)
ax2.legend(loc='center right', fontsize=17)
ax2.tick_params(labelsize=17)
ax2.locator_params(axis='x', nbinx=5)
ax2.locator_params(axis='y', nbins=5)
ax2.set_xlim(0, 3.5)
ax2.set_ylim(-0.98, -0.8)

In [ ]:
with open("data_phase_bound.p", 'wb') as fp:
    pickle.dump(Ez_topo_dict, fp)
    pickle.dump(V_G_range_0V, fp)
    pickle.dump(V_G_range_02V, fp)